In [ ]:
import warnings

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np

import re
import nltk
nltk.download('all')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemm = WordNetLemmatizer()

warnings.filterwarnings("ignore")


In [ ]:
data = pd.read_csv('Clothing Review.csv')
data.head(7)
print(data.shape)

# clean the data
data = data[data['Class Name'].isnull() == False]


In [ ]:
# count plot we automatically plot
# the unique count of the feature selected

sns.countplot(data=data, x='Class Name')
plt.xticks(rotation=90)
plt.show()


In [ ]:
plt.subplots(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.countplot(data=data, x='Rating')

plt.subplot(1, 2, 2)
sns.countplot(data=data, x="Recommended IND")
plt.show()


In [ ]:

fig = px.histogram(data, marginal='box',
				x="Age", title="Age Group",
				color="Recommended IND",
				nbins=65-18,
				color_discrete_sequence=['green', 'red'])
fig.update_layout(bargap=0.2)


In [ ]:
fig = px.histogram(data,
				x="Age",
				marginal='box',
				title="Age Group",
				color="Rating",
				nbins=65-18,
				color_discrete_sequence\
				=['black', 'green', 'blue', 'red', 'yellow'])
fig.update_layout(bargap=0.2)


In [ ]:
def filter_score(rating):
	return int(rating > 3)

features = ['Class Name', 'Title', 'Review Text']

X = data[features]
y = data['Rating']
y = y.apply(filter_score)


In [ ]:
import nltk
nltk.download('stopwords')
def toLower(data):
	if isinstance(data, float):
		return '<UNK>'
	else:
		return data.lower()

stop_words = stopwords.words("english")

def remove_stopwords(text):
	no_stop = []
	for word in text.split(' '):
		if word not in stop_words:
			no_stop.append(word)
	return " ".join(no_stop)

def remove_punctuation_func(text):
	return re.sub(r'[^a-zA-Z0-9]', ' ', text)

# convert into lower case
X['Title'] = X['Title'].apply(toLower)
X['Review Text'] = X['Review Text'].apply(toLower)
# remove common words
X['Title'] = X['Title'].apply(remove_stopwords)
X['Review Text'] = X['Review Text'].apply(remove_stopwords)
# lemmatization
X['Title'] = X['Title'].apply(lambda x: lemm.lemmatize(x))
X['Review Text'] = X['Review Text'].apply(lambda x: lemm.lemmatize(x))
# remove punctuation
X['Title'] = X['Title'].apply(remove_punctuation_func)
X['Review Text'] = X['Review Text'].apply(remove_punctuation_func)

X['Text'] = list(X['Title']+X['Review Text']+X['Class Name'])

# split into training and testing
X_train, X_test, y_train, y_test = train_test_split(
	X['Text'], y, test_size=0.25, random_state=42)


In [ ]:

from tokenize import tokenize


tokenizer = tokenize(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

In [ ]:
from keras.utils.data_utils import pad_sequences
train_seq = tokenizer.texts_to_sequences(X_train)
test_seq = tokenizer.texts_to_sequences(X_test)

train_pad = pad_sequences(train_seq,
						maxlen=40,
						truncating="post",
						padding="post")
test_pad = pad_sequences(test_seq,
						maxlen=40,
						truncating="post",
						padding="post")


In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Embedding(10000, 128))
model.add(keras.layers.SimpleRNN(64, return_sequences=True))
model.add(keras.layers.SimpleRNN(64))
model.add(keras.layers.Dense(128, activation="relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(1, activation="sigmoid"))

model.summary()


In [ ]:
model.compile("rmsprop",
			"binary_crossentropy",
			metrics=["accuracy"])
history = model.fit(train_pad,
					y_train,
					epochs=5)
